## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


In [ ]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
### Helper Functions

def plotImageFiles(filenames):
    for f in filenames:
        fig = plt.figure()
        fig.set_size_inches(3,6)
        img = mpimg.imread(f)
        plt.title(f)
        plt.axis('off')
        plt.imshow(img)
        
def plotImage(img, title=None, cmap=None):
    fig = plt.figure()
    fig.set_size_inches(4,8)
    if title is not None:
        plt.title(title)
    plt.axis('off')
    if cmap is None:
        plt.imshow(img)
    else:
        plt.imshow(img, cmap=cmap)
    
def plotMultipleImages(images, labels=None, ptitle=None, cmap=None):
    """ This function will plot the images specified in a
    single plot.
    """
    numImages = len(images)
    #fig = plt.figure(figsize=(3, 6*numImages))
    fig = plt.figure()
    if ptitle is not None:
        fig.suptitle(ptitle, fontsize="x-large")
    ii = 1
    for img in images:
        ax = fig.add_subplot(1, numImages, ii)
        if labels is not None:
            ax.set_title(labels[ii-1], fontsize="xx-small")
        ax.set_aspect(15)
        plt.axis('off')
        if cmap is not None:
            ax.imshow(img.squeeze(), cmap=cmap)
        else:
            ax.imshow(img.squeeze())
        ii += 1

In [ ]:
### Camera Calibration

def calibrateCamera():
    imfiles = glob.glob('camera_cal/calibration*.jpg')
    nx = 9
    ny = 6
    #3d real world points
    objPoints = []
    #2d camera image points
    imgPoints = []
    #Corners returned by findChessboardCorners contains corners 
    #going left to right and then down. So we have to generate the
    #idealized objpoint to go in the same direction. We also assume
    #that the z-axis is 0 in the objpoint i.e., the image is placed
    #on a 2d plane
    objpoint = np.zeros((nx*ny, 3), np.float32)
    objpoint[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
    for imfile in imfiles:
        img = mpimg.imread(imfile)
        print(imfile)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)
        if ret:
            imgPoints.append(corners)
            objPoints.append(objpoint)
    #ret, mtx, dist, rvecs, tvecs
    return cv2.calibrateCamera(objPoints, imgPoints, (gray.shape[1], gray.shape[0]), None, None)
 
def plotUndistortedImages():
    imfiles = glob.glob('camera_cal/calibration*.jpg')
    ret, mtx, dist, rvecs, tvecs = calibrateCamera()
    for imfile in imfiles:
        img = mpimg.imread(imfile)
        dst = cv2.undistort(img, mtx, dist, None, mtx)
        fig = plt.figure()
        fig.add_subplot(1, 2, 1)
        #plt.axis('off')
        plt.imshow(img)
        fig.add_subplot(1, 2, 2)
        plt.imshow(dst)
        
#plotUndistortedImages()
ret, mtx, dist, rvecs, tvecs = calibrateCamera()

In [ ]:
def getUndistortedImage(img, _mtx = mtx, _dist = dist):
    return cv2.undistort(img, _mtx, _dist, None, _mtx)

def genericSobelThresholding(img, mfunc, ksize=3, thresh=(0,255)):
    # Grayscale transformation if the image has 3 channels
    # Otherwise assume it has already been transformed
    if img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply cv2.Sobel()
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=ksize)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=ksize)
    scaled_sobel = mfunc(sobelx, sobely)
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel > thresh[0]) & (scaled_sobel < thresh[1])] = 1
    return binary_output

def gradientAbsoluteThresholding(img, orient='x', ksize=3, thresh=(0,255)):
    def gradAbsSobel(sobelx, sobely):
        #Calculate scaled value of gradient
        if orient == 'x':
            sobel = sobelx
        else:
            sobel = sobely
        abs_sobel = np.absolute(sobel)
        scaled_sobel = np.uint8((abs_sobel * 255) / np.max(abs_sobel))
        return scaled_sobel
    return genericSobelThresholding(img, gradAbsSobel, ksize, thresh)

def gradientMagnitudeThresholding(img, ksize=3, thresh=(0,255)):
    def gradMagSobel(sobelx, sobely):
        #Calculate scaled magnitude of gradient
        gradmag = np.sqrt(sobelx**2 + sobely**2)
        scaled_gradmag = np.uint8((gradmag * 255) / np.max(gradmag))
        return scaled_gradmag
    return genericSobelThresholding(img, gradMagSobel, ksize, thresh)

def gradientDirectionThresholding(img, ksize=3, thresh=(0,np.pi/2)):
    def gradDirSobel(sobelx, sobely):
        #Calculate direction of gradient
        absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
        return absgraddir
    return genericSobelThresholding(img, gradDirSobel, ksize, thresh)

def colorThresholding(img, channel=0, colspace = cv2.COLOR_RGB2HLS, thresh=(0,255)):
    img = cv2.cvtColor(img, colspace)
    colchannel = img[:,:,channel]
    binary_output = np.zeros_like(colchannel)
    binary_output[(colchannel > thresh[0]) & (colchannel <= thresh[1])] = 1
    return binary_output

In [ ]:
tfiles = glob.glob('test_images/test*.jpg')
for tfile in tfiles:
    img = mpimg.imread(tfile)
    dimg = getUndistortedImage(img)
    colthreshimg = colorThresholding(dimg, channel=2, thresh=(90,255))
    absthreshimg = gradientAbsoluteThresholding(dimg, ksize=5, thresh=(20,100))
    gradmagthreshimg = gradientMagnitudeThresholding(dimg, ksize=5, thresh=(30,100))
    #Direction of gradient == +/- np.pi/2 is a vertical line.
    #We'll never have a fully vertical line at this point due to perspective.
    #So use something smaller than np.pi/2 as max
    graddirthreshimg = gradientDirectionThresholding(dimg, ksize=15, thresh=(0.7,1.3))
    combinedimg = np.zeros_like(colthreshimg)
    #print(colthreshimg)
    combinedimg[((colthreshimg == 1) | (absthreshimg == 1)) & (gradmagthreshimg == 1)] = 1
    images = [dimg, colthreshimg, absthreshimg, gradmagthreshimg, graddirthreshimg, combinedimg]
    labels = ["Undistorted", "S-Channel Color", "Gradient AbsoluteX", "Gradient Magnitude", "Gradient Direction", "Combined"]
    #plotMultipleImages(images, labels, tfile, "gray")
    for i in range(0, len(images)):
        img = images[i]
        lbl = labels[i]
        plotImage(img, lbl, cmap="gray")